In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = 4
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}



In [3]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 111 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 114 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [4]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [5]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [6]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:00+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_2...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading NILMTK mains data.
INFO:neuralnilm.data.realaggregatesource:Found 89 sections without targ

In [65]:
ras.target_inclusion_prob = 0.5
for i in range(50):
    seq = ras.get_sequence()

In [66]:
fig, axes = plt.subplots(2)
axes[0].plot(seq.input)
axes[1].plot(seq.target)
plt.show()

In [42]:
seq.input

array([[ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ n

In [43]:
ras.mains['train']['UK-DALE_building_1']["2014-01-17 23:30:38+00:00":"2014-01-17 23:56:08+00:00"].values

array([ 468.44500732,  469.13500977,  470.48666382,  471.19500732,
        471.59667969,  471.79333496,  471.86999512,  472.91665649,
        476.1416626 ,  474.70999146,  474.77001953,  474.82833862,
        474.69500732,  474.35501099,  474.56668091,  474.58001709,
        475.15667725,  474.66998291,  473.91998291,  473.91833496,
        473.84500122,  474.22332764,  475.80166626,  474.04333496,
        473.97167969,  475.80999756,  475.34832764,  476.05999756,
        475.91833496,  475.6166687 ,  475.30334473,  475.82833862,
        475.95999146,  475.51165771,  477.33834839,  478.37167358,
        477.7583313 ,  478.06665039,  479.24334717,  477.08428955,
        477.29199219,  483.67001343,  476.4750061 ,  477.8999939 ,
        487.23666382,  487.46331787,  477.61831665,  486.46331787,
        494.03500366,  498.07833862,  500.58831787,  492.77331543,
        489.91333008,  492.21499634,  489.69168091,  493.57501221,
        493.38833618,  493.60501099,  493.70834351,  482.50500

In [4]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [5]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0].building

'UK-DALE_building_1'

In [6]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [7]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]

2014-01-01 10:40:12+00:00       1
2014-01-01 10:40:18+00:00       1
2014-01-01 10:40:24+00:00    2371
2014-01-01 10:40:30+00:00    2371
2014-01-01 10:40:36+00:00    2322
2014-01-01 10:40:42+00:00    2301
2014-01-01 10:40:48+00:00    2300
2014-01-01 10:40:54+00:00    2332
2014-01-01 10:41:00+00:00    2340
2014-01-01 10:41:06+00:00    2328
2014-01-01 10:41:12+00:00    2298
2014-01-01 10:41:18+00:00    2290
2014-01-01 10:41:24+00:00    2299
2014-01-01 10:41:30+00:00    2289
2014-01-01 10:41:36+00:00    2289
2014-01-01 10:41:42+00:00    2289
2014-01-01 10:41:48+00:00    2309
2014-01-01 10:41:54+00:00    2318
2014-01-01 10:42:00+00:00    2307
2014-01-01 10:42:06+00:00    2310
2014-01-01 10:42:12+00:00    2305
2014-01-01 10:42:18+00:00    2308
2014-01-01 10:42:24+00:00    2325
2014-01-01 10:42:30+00:00       0
Freq: 6S, Name: (power, active), dtype: float32

In [4]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False
)

In [5]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [6]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [7]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [8]:
sample = source.get_batch(num_seq_per_batch=1024)

In [9]:
pipeline = DataPipeline(
    [source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std())],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [10]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
   'allow_incomplete_target': False,
   'distractor_inclusion_prob': 1.0,
   'include_incomplete_target_in_output': True,
   'rng_seed': None,
   'seq_length': 256,
   'target_appliance': 'kettle',
   'target_inclusion_prob': 1.0,
   'uniform_prob_of_selecting_each_model': True},
  'input_processing': [{'DivideBy': {'divisor': 987.1983}}],
  'num_seq_per_batch': 64,
  'target_processing': [{'DivideBy': {'divisor': 600.25781}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'windows': {'train': {1: ('2014-01-01', '2014-02-01')},
    'unseen_activations_of_seen_appliances': {1: ('2014-02-02', '2014-02-08')},
    'unseen_appliances': {2: ('2013-06-01', '2013-06-07')}}}}}

In [11]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [12]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [13]:
from pymongo import MongoClient
client = MongoClient()
client.drop_database('neuralnilm_experiments')
db = client.neuralnilm_experiments

In [14]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=0,
    metrics=Metrics(state_boundaries=[4]),
    learning_rates={0: 1E-2},
    repeat_callbacks=[
        ( 100, Trainer.validate)
    ]
)

INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02


In [15]:
trainer.fit(400)

INFO:neuralnilm.trainer:Starting training for 400 iterations.
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.
INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Stopped training. Completed 400 iterations.


In [16]:
net.train_iterations

400

In [17]:
import pymongo
list(db.train_scores.find(
        filter={'experiment_id': 0},
        sort=[('iteration', pymongo.ASCENDING)],
        projection=['iteration', 'loss']
    ).limit(50))

[{u'_id': ObjectId('55e00adb42c36467c1ea54c1'),
  u'iteration': 0,
  u'loss': 1.4566963030431732},
 {u'_id': ObjectId('55e00adb42c36467c1ea54c5'),
  u'iteration': 1,
  u'loss': 1.504454061303818},
 {u'_id': ObjectId('55e00adb42c36467c1ea54c6'),
  u'iteration': 2,
  u'loss': 1.634689091431984},
 {u'_id': ObjectId('55e00adb42c36467c1ea54c7'),
  u'iteration': 3,
  u'loss': 1.5483386156339893},
 {u'_id': ObjectId('55e00adb42c36467c1ea54c8'),
  u'iteration': 4,
  u'loss': 1.4937821382597245},
 {u'_id': ObjectId('55e00adb42c36467c1ea54c9'),
  u'iteration': 5,
  u'loss': 1.3925116251212233},
 {u'_id': ObjectId('55e00adb42c36467c1ea54ca'),
  u'iteration': 6,
  u'loss': 1.2735618170789764},
 {u'_id': ObjectId('55e00adb42c36467c1ea54cb'),
  u'iteration': 7,
  u'loss': 1.3696621122441406},
 {u'_id': ObjectId('55e00adb42c36467c1ea54cc'),
  u'iteration': 8,
  u'loss': 1.2664201029716082},
 {u'_id': ObjectId('55e00adb42c36467c1ea54cd'),
  u'iteration': 9,
  u'loss': 1.149360545211264},
 {u'_id': Obj

In [18]:
list(db.validation_scores.find({'fold': 'train', 'metric_type': 'classification'}).sort('iteration', pymongo.ASCENDING).limit(50))

[]

In [19]:
agg_pipeline = [
    {"$group": {
            "_id": "$experiment_id",
            "max_acc": {"$max": "$SyntheticAggregateSource.unseen_appliances.2_state_classification.accuracy_score"},
            "max_iteration": {"$max": "$iteration"}
        }}
]
list(db.validation_scores.aggregate(agg_pipeline))

[{u'_id': 0, u'max_acc': None, u'max_iteration': 400}]

In [20]:
from monary import Monary

In [21]:
monary = Monary()

In [22]:
iterations, loss, source_id = monary.query(
    db='neuralnilm_experiments',
    coll='train_scores',
    query={
        'experiment_id': 0,
        'iteration': {'$gt': 0}
    },
    fields=['iteration', 'loss', 'source_id'],
    types=['int32', 'float32', 'int8']
)

In [23]:
import pandas as pd

df = pd.DataFrame({'loss': loss, 'source_id': source_id}, index=iterations)

In [24]:
import matplotlib.pyplot as plt
plt.plot(df.index, df['loss'].values)
plt.show()

In [25]:
from neuralnilm.monitor.monitor import Monitor

mon = Monitor(0)

In [26]:
mon._plot_train_scores()

In [27]:
mon._plot_validation_scores()

In [28]:
mon.validation_metric_names

[u'regression.mean_squared_error',
 u'regression.relative_error_in_total_energy',
 u'regression.mean_absolute_error',
 u'classification_2_state.f1_score',
 u'classification_2_state.recall_score',
 u'classification_2_state.accuracy_score',
 u'classification_2_state.precision_score']